# Shelter Animal Outcomes

In [41]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import math

In [2]:
os.chdir(r'C:\Users\brcro\OneDrive\Documents\Syracuse\IST 652 - Scripting for Data Analysis\Project')

In [88]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
train.shape

(26729, 9)

In [89]:
train = train.set_index('AnimalID')
train.head()

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
AnimalID,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


In [7]:
train.columns

Index(['Name', 'DateTime', 'OutcomeType', 'OutcomeSubtype', 'AnimalType',
       'SexuponOutcome', 'AgeuponOutcome', 'Breed', 'Color'],
      dtype='object')

In [8]:
train.describe()

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
count,19038,26729,26729,13117,26729,26728,26711,26729,26729
unique,6374,22918,5,16,2,5,44,1380,366
top,Max,2015-08-11 00:00:00,Adoption,Partner,Dog,Neutered Male,1 year,Domestic Shorthair Mix,Black/White
freq,136,19,10769,7816,15595,9779,3969,8810,2824


In [10]:
train.dtypes

Name              object
DateTime          object
OutcomeType       object
OutcomeSubtype    object
AnimalType        object
SexuponOutcome    object
AgeuponOutcome    object
Breed             object
Color             object
dtype: object

In [90]:
#Replace Missing Values
train = train.fillna('Unknown')

In [62]:
#Change data types
train['Name'] = str(train['Name'])
train['OutcomeType'] = train['OutcomeType'].astype('category')
train['OutcomeSubtype'] = train['OutcomeSubtype'].astype('category')
train['AnimalType'] = train['AnimalType'].astype('category')
train['SexuponOutcome'] = str(train['SexuponOutcome'])
train['AgeuponOutcome'] = str(train['AgeuponOutcome'])
train['Breed'] = train['Breed'].astype('category')
train['Color'] = train['Color'].astype('category')

In [63]:
train.dtypes

AnimalID            object
Name                object
DateTime            object
OutcomeType       category
OutcomeSubtype    category
AnimalType        category
SexuponOutcome      object
AgeuponOutcome      object
Breed             category
Color             category
dtype: object

## Feature Engineering

In [17]:
train.head()

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
AnimalID,,,,,,,,,
A671945,AnimalID\nA671945 AnimalID\nA671945 Anim...,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
A656520,AnimalID\nA671945 AnimalID\nA671945 Anim...,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
A686464,AnimalID\nA671945 AnimalID\nA671945 Anim...,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
A683430,AnimalID\nA671945 AnimalID\nA671945 Anim...,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
A667013,AnimalID\nA671945 AnimalID\nA671945 Anim...,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


In [91]:
def NG_split(x):
    if x != 'Unknown':
        Neutered, Gender = x.split()
        if Neutered == 'Intact':
            Neutered == False
        else:
            Neutered == True
    else:
        Neutered = None
        Gender = None
    return(Neutered, Gender)

In [92]:
train['SexuponOutcome'].unique()

array(['Neutered Male', 'Spayed Female', 'Intact Male', 'Intact Female',
       'Unknown'], dtype=object)

In [93]:
Neutered = []
Gender = []
for val in train['SexuponOutcome']:
    neutered, gender = NG_split(val)
    Neutered.append(neutered)
    Gender.append(gender)
train['Neutered'] = Neutered
train['Gender'] = Gender
train = train.drop('SexuponOutcome', 1)
train.head()

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,AgeuponOutcome,Breed,Color,Neutered,Gender
AnimalID,,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,Unknown,Dog,1 year,Shetland Sheepdog Mix,Brown/White,Neutered,Male
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,1 year,Domestic Shorthair Mix,Cream Tabby,Spayed,Female
A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,2 years,Pit Bull Mix,Blue/White,Neutered,Male
A683430,Unknown,2014-07-11 19:09:00,Transfer,Partner,Cat,3 weeks,Domestic Shorthair Mix,Blue Cream,Intact,Male
A667013,Unknown,2013-11-15 12:52:00,Transfer,Partner,Dog,2 years,Lhasa Apso/Miniature Poodle,Tan,Neutered,Male


In [95]:
train['AgeuponOutcome'].unique()

array(['1 year', '2 years', '3 weeks', '1 month', '5 months', '4 years',
       '3 months', '2 weeks', '2 months', '10 months', '6 months',
       '5 years', '7 years', '3 years', '4 months', '12 years', '9 years',
       '6 years', '1 weeks', '11 years', '4 weeks', '7 months', '8 years',
       '11 months', '4 days', '9 months', '8 months', '15 years',
       '10 years', '1 week', '0 years', '14 years', '3 days', '6 days',
       '5 days', '5 weeks', '2 days', '16 years', '1 day', '13 years',
       'Unknown', '17 years', '18 years', '19 years', '20 years'],
      dtype=object)

In [108]:
def agetodays(x):
        try:
            if x == 'Unknown':
                return None
            else:
                y = x.split()
        except:
            return None 
        if 'year' in y[1]:
            return float(y[0]) * 365
        elif 'month' in y[1]:
            return float(y[0]) * (365/12)
        elif 'week' in y[1]:
            return float(y[0]) * 7
        elif 'day' in y[1]:
            return float(y[0])
        
train['Age'] = train['AgeuponOutcome'].map(agetodays)
train.head()

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,AgeuponOutcome,Breed,Color,Neutered,Gender,Age
AnimalID,,,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,Unknown,Dog,1 year,Shetland Sheepdog Mix,Brown/White,Neutered,Male,365.0
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,1 year,Domestic Shorthair Mix,Cream Tabby,Spayed,Female,365.0
A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,2 years,Pit Bull Mix,Blue/White,Neutered,Male,730.0
A683430,Unknown,2014-07-11 19:09:00,Transfer,Partner,Cat,3 weeks,Domestic Shorthair Mix,Blue Cream,Intact,Male,21.0
A667013,Unknown,2013-11-15 12:52:00,Transfer,Partner,Dog,2 years,Lhasa Apso/Miniature Poodle,Tan,Neutered,Male,730.0


In [111]:
train['Year'] = train['DateTime'].str[:4].astype(int)
train['Month'] = train['DateTime'].str[5:7].astype(int)
train['Day'] = train['DateTime'].str[8:10].astype(int)
train.drop('DateTime', 1)

,Name,OutcomeType,OutcomeSubtype,AnimalType,AgeuponOutcome,Breed,Color,Neutered,Gender,Age,Year,Month,Day,Hour,Minute
AnimalID,,,,,,,,,,,,,,,
A671945,Hambone,Return_to_owner,Unknown,Dog,1 year,Shetland Sheepdog Mix,Brown/White,Neutered,Male,365.000000,2014,2,12,18,22
A656520,Emily,Euthanasia,Suffering,Cat,1 year,Domestic Shorthair Mix,Cream Tabby,Spayed,Female,365.000000,2013,10,13,12,44
A686464,Pearce,Adoption,Foster,Dog,2 years,Pit Bull Mix,Blue/White,Neutered,Male,730.000000,2015,1,31,12,28
A683430,Unknown,Transfer,Partner,Cat,3 weeks,Domestic Shorthair Mix,Blue Cream,Intact,Male,21.000000,2014,7,11,19,9
A667013,Unknown,Transfer,Partner,Dog,2 years,Lhasa Apso/Miniature Poodle,Tan,Neutered,Male,730.000000,2013,11,15,12,52
A677334,Elsa,Transfer,Partner,Dog,1 month,Cairn Terrier/Chihuahua Shorthair,Black/Tan,Intact,Female,30.416667,2014,4,25,13,4
A699218,Jimmy,Transfer,Partner,Cat,3 weeks,Domestic Shorthair Mix,Blue Tabby,Intact,Male,21.000000,2015,3,28,13,11
A701489,Unknown,Transfer,Partner,Cat,3 weeks,Domestic Shorthair Mix,Brown Tabby,None,None,21.000000,2015,4,30,17,2
A671784,Lucy,Adoption,Unknown,Dog,5 months,American Pit Bull Terrier Mix,Red/White,Spayed,Female,152.083333,2014,2,4,17,17


In [112]:
train['OutcomeType'].unique()

array(['Return_to_owner', 'Euthanasia', 'Adoption', 'Transfer', 'Died'],
      dtype=object)

In [109]:
colors = ['Brown', 'White', 'Blue', 'White', 'Tan', 'Black', 'Chocolate', 'Red', 'Yellow', 'Calico']
train['Color'].unique()

array(['Brown/White', 'Cream Tabby', 'Blue/White', 'Blue Cream', 'Tan',
       'Black/Tan', 'Blue Tabby', 'Brown Tabby', 'Red/White', 'White',
       'Black', 'Silver', 'Brown', 'Black/Red', 'White/Cream',
       'Orange Tabby/White', 'Black/White', 'Brown Brindle/White',
       'Black/Brown', 'Orange Tabby', 'Chocolate/White', 'White/Tan',
       'Cream Tabby/White', 'Blue', 'Calico', 'Torbie', 'Brown/Black',
       'Yellow', 'Tricolor', 'White/Black', 'Tortie', 'Blue Tabby/White',
       'Gray/White', 'Tan/Black', 'Tan/White', 'Buff',
       'Brown Tabby/White', 'Red', 'Blue/Tan', 'Seal Point',
       'Brown Brindle', 'White/Brown', 'Gray', 'Yellow Brindle/White',
       'Fawn/White', 'Flame Point', 'Black Tabby/White', 'Tortie/Calico',
       'Black/Brown Brindle', 'White/Gray', 'Tan/Silver',
       'Red Tick/Black', 'White/Cream Tabby', 'Blue Merle',
       'Chocolate/Tan', 'Sable', 'Brown Merle/White', 'Brown Tiger/White',
       'Liver/White', 'White/Blue Tabby', 'Black/Gray', 'C

(array([], dtype=int64),)